# U Net Arch

In [ ]:
# -*- coding: utf-8 -*-
# Adam Optimizer
# 
"""
Created on Thu Feb 13 22:24:51 2024

@author: Dheepak
"""


from keras.models import Model
from keras.layers import Input, Conv2DTranspose, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda




################################################################
def multi_unet_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    #model.summary()
    
    return model
 

In [ ]:
import cv2
import glob

SIZE_X = 128
SIZE_Y = 128

train_images = []

image_paths = glob.glob("E:/MICCAI Dataset/Training set/Images/*.png")
for img_path in image_paths:
    img = cv2.imread(img_path, 0)
    img = cv2.resize(img, (SIZE_Y, SIZE_X))

    if img is None:
        print(f"Failed to load image: {img_path}")
        continue

    filename = (img_path.split("Images\\")[1]).split(".")[0]
    masks_path = f"E:/MICCAI Dataset/Training set/Masks/{filename}.png"
    
    masks = cv2.imread(masks_path, 0)
    masks = cv2.resize(masks, (SIZE_Y, SIZE_X))

    if masks is None:
        print(f"Failed to load mask: {masks_path}")
        continue

    # Append the image and mask to the list
    train_images.append(img)
    train_images.append(masks)

# Check the length of the list
print(len(train_images))


In [ ]:
train_images = np.array(train_images)

In [ ]:
print(len(train_images))

In [ ]:
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

train_masks = [] 
for directory_path in glob.glob("E:\\MICCAI Dataset\\Training Set\\Masks"):
    #print('hi')
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (128, 128), interpolation = cv2.INTER_NEAREST) 
        masks_arr = np.array(mask)
        length = len(np.unique(masks_arr))
        #if len(np.unique(masks_arr))==4:
            #print(np.unique(masks_arr), mask_path)
          
        if length ==4:
            mask[mask==0] = 1
            mask[mask==85] = 2 
            mask[mask==170] = 3
            mask[mask==255] = 4
        elif length==3:
            mask[mask==0] = 1
            if 85 in np.unique(masks_arr):
                mask[mask==85]=2
                mask[mask==170]=3
            elif 170 in np.unique(masks_arr):
                mask[mask==170]= 2
                mask[mask==255]= 4
            else:
                mask[mask==127]= 2
                mask[mask==255]= 3
        elif length==2:
            mask[mask==0] = 1
            mask[mask==255] = 2
        else:
            mask[mask==0] = 1
        #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
print(len(train_masks))

In [ ]:
np.unique(train_masks)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)


In [ ]:
np.unique(train_masks_encoded_original_shape)
train_images = np.expand_dims(train_images, axis=3)

In [ ]:
train_images = normalize(train_images, axis=1)
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

In [ ]:
print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 


In [ ]:
from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))


In [ ]:

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))


In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

In [ ]:
print(X_train.shape)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    callbacks=[early_stopping],
                    #class_weight=class_weights,
                    shuffle=False)

In [ ]:
model.save('categorical_1.hdf5')
#model.save('sandstone_50_epochs_catXentropy_acc_with_weights.hdf5')
############################################################
#Evaluate the model
	# evaluate model
_, acc = model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])


plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
model = get_model()
model.load_weights('categorical_2.hdf5')  
#model.load_weights('sandstone_50_epochs_catXentropy_acc_with_weights.hdf5')  

#IOU
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
from keras.metrics import MeanIoU
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
import random
test_img_number =  random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test_cat[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='jet')
plt.show()

In [ ]:
pip install tensorflow-addons

In [ ]:
import pandas as pd
import tensorflow as tf
#import tensorflow_addons as tfa


#Calculate IoU for all test images and average
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

In [ ]:
jaccards=[]
def jacard_coef(y_true, y_pred):
    
    smooth=1
    intersection=K.sum(K.abs(y_true*y_pred),axis=-1)
    union=K.sum(y_true,-1)+K.sum(y_pred,-1)-intersection
    iou=K.mean((intersection+smooth)/(union+smooth),axis=0)
    return iou

In [ ]:
def jacard_coef_loss(y_true, y_pred):
    jacard=0
    for i in range(4):
    
        jacard=jacard_coef(y_true[:,:,:,i], y_pred[:,:,:,i])
        jaccards.append(jacard)
    return np.mean(jaccards)

In [ ]:
dices=[]
def dice_coef(y_true, y_pred):
    y_true = K.cast(y_true, 'float64')
    y_pred = K.cast(y_pred, 'float64')
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)


In [ ]:
model

## Validation and Testing

In [ ]:
import cv2
import os
import glob
import numpy as np

SIZE_X = 128
SIZE_Y = 128

def load_images_and_masks(directory_path):
    images = []
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        images.append(img)
    return images

def load_masks_and_convert(directory_path):
    masks = []
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation=cv2.INTER_NEAREST)
        masks_arr = np.array(mask)
        length = len(np.unique(masks_arr))
        if length == 4:
            mask[mask == 0] = 1
            mask[mask == 85] = 2
            mask[mask == 170] = 3
            mask[mask == 255] = 4
        elif length == 3:
            mask[mask == 0] = 1
            if 85 in np.unique(masks_arr):
                mask[mask == 85] = 2
                mask[mask == 170] = 3
            elif 170 in np.unique(masks_arr):
                mask[mask == 170] = 2
                mask[mask == 255] = 4
            else:
                mask[mask == 127] = 2
                mask[mask == 255] = 3
        elif length == 2:
            mask[mask == 0] = 1
            mask[mask == 255] = 2
        else:
            mask[mask == 0] = 1
        masks.append(mask)
    return masks






In [ ]:
# Load testing images and masks
test_images = load_images_and_masks("E:/MICCAI Dataset/Testing Set/Test_images")
test_masks = load_masks_and_convert("E:/MICCAI Dataset/Testing Set/Test_masks")
test_images = np.array(test_images)
test_masks = np.array(test_masks)

In [ ]:

# Load validation images and masks
val_images = load_images_and_masks("E:/MICCAI Dataset/Validation Set/Validation_Images")
val_masks = load_masks_and_convert("E:/MICCAI Dataset/Validation Set/Validation_Masks")
val_images = np.array(val_images)
val_masks = np.array(val_masks)

In [ ]:
# Reshape masks to 1D arrays
y_val_flat = val_masks.flatten()
y_test_flat = test_masks.flatten()

y_val_flat[y_val_flat == 1] = 0
y_val_flat[y_val_flat == 2] = 1
y_val_flat[y_val_flat == 3] = 2
y_val_flat[y_val_flat == 4] = 3

y_test_flat[y_test_flat == 1] = 0
y_test_flat[y_test_flat == 2] = 1
y_test_flat[y_test_flat == 3] = 2
y_test_flat[y_test_flat == 4] = 3

# Convert to one-hot encoded format
y_val_one_hot = to_categorical(y_val_flat, num_classes=4)
y_test_one_hot = to_categorical(y_test_flat, num_classes=4)

# Reshape back to original shape if needed
y_val_one_hot = y_val_one_hot.reshape(val_masks.shape[0], val_masks.shape[1], val_masks.shape[2], 4)
y_test_one_hot = y_test_one_hot.reshape(test_masks.shape[0], test_masks.shape[1], test_masks.shape[2], 4)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_images, y_val_one_hot, verbose=0)
test_loss, test_accuracy = model.evaluate(test_images, y_test_one_hot, verbose=0)

print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')


In [ ]:
print(X_test.shape)
print(y_test_one_hot.shape)


In [ ]:
import tensorflow.keras.backend as K
K.clear_session()


In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

output_path = 'E:/MICCAI Dataset/Predicted_masks'

# Function to calculate Dice coefficient
def dice_coef(y_true, y_pred):
    smooth = 1e-5
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

# Loop over each image in the test set
for img in range(0, X_test.shape[0]):
    temp_img = X_test[img]
    ground_truth1 = y_test_cat[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    ground_truth = np.expand_dims(ground_truth1, 0)
    prediction = (model.predict(temp_img_input)).astype('float32')
    print(dice_coef(ground_truth, prediction))
    predicted_img = np.argmax(prediction, axis=3)[0,:,:]
    
    plt.figure(figsize=(18, 12))

    # Plot original image
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(temp_img)

    # Plot ground truth label for each class
    for i in range(ground_truth1.shape[-1]):
        plt.subplot(2, 3, i+2)
        plt.title(f'Ground Truth Class {i}')
        plt.imshow(ground_truth1[:,:,i], cmap='gray')

    # Plot predicted image
    plt.subplot(236)
    plt.title('Predicted Image')
    plt.imshow(predicted_img, cmap='gray')

    plt.show()

In [ ]:
import os

# Assuming the 'E:/MICCAI Dataset/Predicted_masks' directory exists

# Path to save the predicted masks
output_path = 'E:/MICCAI Dataset/Predicted_masks'

# Loop over each image in the test set
for img in range(0, test_images.shape[0]):
    temp_img = test_images[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    prediction = model.predict(temp_img_input).astype('float32')
    predicted_img = np.argmax(prediction, axis=3)[0,:,:]
    
    plt.figure(figsize=(18, 12))

    # Plot original image
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(temp_img)

    # Plot predicted image
    plt.subplot(236)
    plt.title('Predicted Image')
    plt.imshow(predicted_img, cmap='gray')

    # Save predicted image
    image_name = f'predicted_{img}.png'
    image_path = os.path.join(output_path, image_name)
    plt.imsave(image_path, predicted_img, cmap='gray')

    plt.show()

In [ ]:
for img in range(0, X_test.shape[0]):
    temp_img = X_test[img]
    ground_truth1 = y_test_cat[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    ground_truth = np.expand_dims(ground_truth1, 0)
    prediction = (model.predict(temp_img_input)).astype('float32')
    print(dice_coef(ground_truth, prediction))
    predicted_img = np.argmax(prediction, axis=3)[0,:,:]
    
    plt.figure(figsize=(18, 12))

    # Plot original image
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(temp_img)

    # Plot ground truth label for each class
    for i in range(ground_truth1.shape[-1]):
        plt.subplot(2, 3, i+2)
        plt.title(f'Ground Truth Class {i}')
        plt.imshow(ground_truth1[:,:,i], cmap='gray')

    # Plot predicted image
    plt.subplot(236)
    plt.title('Predicted Image')
    plt.imshow(predicted_img, cmap='gray')

    plt.show()


In [ ]:
from sklearn.metrics import jaccard_score

y_true_indicator = tf.keras.utils.to_categorical(y_test_cat, num_classes=4)
y_pred_indicator = tf.keras.utils.to_categorical(np.argmax(y_pred, axis=3), num_classes=4)

# Calculate Jaccard score
jacard = jaccard_score(y_true_indicator.reshape(-1), y_pred_indicator.reshape(-1), average='macro')
print(jacard)

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

def dice_loss(y_true, y_pred, smooth=1e-5):
    intersection = np.sum(y_true * y_pred, axis=(1,2,3))
    sum_of_squares_pred = np.sum(np.square(y_pred), axis=(1,2,3))
    sum_of_squares_true = np.sum(np.square(y_true), axis=(1,2,3))
    dice = 1 - (2 * intersection + smooth) / (sum_of_squares_pred + sum_of_squares_true + smooth)
    return dice

In [ ]:
model = get_model()

# Compile the model with focal loss
model.compile(optimizer='adam', loss=dice_loss(y_true,y_pred), metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    callbacks=[early_stopping],
                    #class_weight=class_weights,
                    shuffle=False)


# Focal Loss

In [ ]:
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # Calculate cross entropy for each channel
        cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=True)
        
        # Calculate the modulating factor for each channel
        y_pred_prob = tf.nn.softmax(y_pred, axis=-1)
        p_t = tf.reduce_sum(y_true * y_pred_prob, axis=-1)
        modulating_factor = (1.0 - p_t)**gamma
        
        # Calculate the final focal loss for each channel
        focal_loss = alpha * modulating_factor * cross_entropy
        
        # Average the loss over all channels
        return tf.reduce_mean(focal_loss)
    
    return focal_loss_fixed

In [ ]:
# Get the model
model = get_model()

# Compile the model with focal loss
model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    #class_weight=class_weights,
                    shuffle=False)

In [ ]:
model.save('focal_1.hdf5')
#model.save('sandstone_50_epochs_catXentropy_acc_with_weights.hdf5')
############################################################
#Evaluate the model
	# evaluate model
_, acc = model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])


plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model = get_model()
model.load_weights('focal_1.hdf5')  
#model.load_weights('sandstone_50_epochs_catXentropy_acc_with_weights.hdf5')  

#IOU
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

## Hausdorff Distance

In [ ]:
import os
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from skimage import io
from collections import defaultdict

# Path to testing masks and predicted masks
test_mask_path = 'E:/MICCAI Dataset/Testing Set/Test_masks'
predicted_mask_path = 'E:/MICCAI Dataset/Predicted_masks'

# Mapping of class labels to class names
class_names = {0: 'BG', 1: 'LV', 2: 'MYO', 3: 'RV'}

In [ ]:
import os
import numpy as np
from skimage import io, transform
import cv2

import scipy.ndimage

def hausdorff_distance(image1, image2):
    """
    Calculate the Hausdorff distance between two multi-class images.

    Args:
    - image1 (numpy.ndarray): The first multi-class image.
    - image2 (numpy.ndarray): The second multi-class image.

    Returns:
    - float: The Hausdorff distance between the two images.
    """
    distances = []
    for label in range(0, 4):  # Assuming 4 classes
        mask1 = (image1 == label)
        mask2 = (image2 == label)
        if np.sum(mask1) == 0 or np.sum(mask2) == 0:
            continue
        distance = scipy.ndimage.morphology.distance_transform_edt(~mask1)
        distance_transformed = distance[mask2]
        distances.append(np.max(distance_transformed))
    if len(distances) == 0:
        return 0
    return np.max(distances)

def label_test_mask(mask):
    mask_resized = mask.astype(np.uint8)
    
    length = len(np.unique(mask_resized))
    if length == 4:
        mask_resized[mask_resized == 0] = 0
        mask_resized[mask_resized == 85] = 1
        mask_resized[mask_resized == 170] = 2
        mask_resized[mask_resized == 255] = 3
    elif length == 3:
        mask_resized[mask_resized == 0] = 0
        if 85 in np.unique(mask_resized):
            mask_resized[mask_resized == 85] = 1
            mask_resized[mask_resized == 170] = 2
        elif 170 in np.unique(mask_resized):
            mask_resized[mask_resized == 170] = 1
            mask_resized[mask_resized == 255] = 3
        else:
            mask_resized[mask_resized == 127] = 1
            mask_resized[mask_resized == 255] = 2
    elif length == 2:
        mask_resized[mask_resized == 0] = 0
        mask_resized[mask_resized == 255] = 1
    else:
        mask_resized[mask_resized == 0] = 0
    
    return mask_resized


def resize_and_label_mask(mask):
    mask_resized = transform.resize(mask, (128, 128), order=0, anti_aliasing=False, preserve_range=True)
    mask_resized = mask_resized.astype(np.uint8)
    
    length = len(np.unique(mask_resized))
    if length == 4:
        mask_resized[mask_resized == 0] = 0
        mask_resized[mask_resized == 85] = 1
        mask_resized[mask_resized == 170] = 2
        mask_resized[mask_resized == 255] = 3
    elif length == 3:
        mask_resized[mask_resized == 0] = 0
        if 85 in np.unique(mask_resized):
            mask_resized[mask_resized == 85] = 1
            mask_resized[mask_resized == 170] = 2
        elif 170 in np.unique(mask_resized):
            mask_resized[mask_resized == 170] = 1
            mask_resized[mask_resized == 255] = 3
        else:
            mask_resized[mask_resized == 127] = 1
            mask_resized[mask_resized == 255] = 2
    elif length == 2:
        mask_resized[mask_resized == 0] = 0
        mask_resized[mask_resized == 255] = 1
    else:
        mask_resized[mask_resized == 0] = 0
    
    return mask_resized

# Paths to directories
test_images_dir = 'E:/MICCAI Dataset/Testing Set/Test_images'
test_masks_dir = 'E:/MICCAI Dataset/Testing Set/Test_masks'
predicted_masks_dir = 'E:/MICCAI Dataset/Predicted_masks'

# List files in test_masks_dir
test_mask_files = os.listdir(test_masks_dir)

# Initialize Hausdorff distances list
hausdorff_distances = []

for test_mask_file in test_mask_files:
    # Load test mask and resize
    test_mask_path = os.path.join(test_masks_dir, test_mask_file)
    test_mask = cv2.imread(test_mask_path)
    test_mask_resized = transform.resize(test_mask, (128, 128), order=0, anti_aliasing=False, preserve_range=True)
    test_mask_resized = test_mask_resized.astype(np.uint8)
    
    # Label test mask
    test_mask_resized_labeled = label_test_mask(test_mask_resized)

    # Load corresponding test image
    test_image_path = os.path.join(test_images_dir, test_mask_file.replace('_mask', ''))
    test_image = io.imread(test_image_path)

    # Extract index from test_mask_file to load corresponding predicted mask
    index = int(test_mask_file.split('_')[-1].split('.')[0])
    predicted_mask_name = f"predicted_{index}"
    predicted_mask_path = os.path.join(predicted_masks_dir, predicted_mask_name + '.png')
    predicted_mask = cv2.imread(predicted_mask_path)

    
    predicted_mask = transform.resize(predicted_mask, (128, 128), order=0, anti_aliasing=False, preserve_range=True)
    predicted_mask_resized = label_test_mask(predicted_mask)

    # Calculate and append Hausdorff distance
    hausdorff_distance_value = hausdorff_distance(test_mask_resized, predicted_mask_resized)
    #print(f"Hausdorff distance for {test_mask_file}: {hausdorff_distance_value}")
    hausdorff_distances.append(hausdorff_distance_value)


# Calculate mean Hausdorff distance
mean_hausdorff_distance = np.mean(hausdorff_distances)

# Print mean Hausdorff distance
print(f"Mean Hausdorff Distance: {mean_hausdorff_distance}")


# Jaccord Coefficient

In [ ]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

# Dice Index

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)